<a href="https://colab.research.google.com/github/louisowen6/Text_Data_Cleaning/blob/master/Text_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [1]:
!pip install contractions

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import numpy as np
import re
import json
import math
from bs4 import BeautifulSoup
import inflect
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import wordpunct_tokenize
import contractions
import string

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Mount the Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Importing Data

In [5]:
link ='https://drive.google.com/open?id=1szYV-zxOuHUWf2RlIKuvZD3qLccRmKO-'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Microblog_Train_twitter_full.json')  
with open('Microblog_Train_twitter_full.json') as train_file:
    dict_train = json.load(train_file)
microblog_train_twitter_full = pd.DataFrame.from_dict(dict_train)

1szYV-zxOuHUWf2RlIKuvZD3qLccRmKO-


In [6]:
link ='https://drive.google.com/open?id=1hsmjwP7FiDAkq4hgWbjVTUJ9fY1yKEyc'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Microblog_Train_stocktwits_full.json')  
with open('Microblog_Train_stocktwits_full.json') as train_file:
    dict_train = json.load(train_file)
microblog_train_stocktwits_full = pd.DataFrame.from_dict(dict_train)

1hsmjwP7FiDAkq4hgWbjVTUJ9fY1yKEyc


In [7]:
#Abbreviation & Slang Dict
link ='https://drive.google.com/open?id=1ptzEljjFTQNF3FRR9wO9G7D0I9DWwjQR'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('abbreviation_dict.txt')  
abb_df = pd.read_table('abbreviation_dict.txt', sep='\s+', names=('Abbreviation', 'Normal'))
abb=pd.Series(abb_df['Normal'])
abb.index=abb_df['Abbreviation']
abb_dict=dict(abb)

1ptzEljjFTQNF3FRR9wO9G7D0I9DWwjQR


# Rebuild Data

In [0]:
twitter_df=microblog_train_twitter_full[['cashtag','sentiment score','spans','text','created_at']].copy()
twitter_df['source']=['twitter' for x in range(len(twitter_df))] #add source column
twitter_df['created_at']=pd.to_datetime(twitter_df['created_at']).apply(lambda x: '[0am,9am)' 
                                                                        if (x.hour>=0 and x.hour<9) else '[9am,3pm)' 
                                                                        if (x.hour>=9 and x.hour<15) else '[3pm,24pm)')  #encoding time created

In [0]:
stocktwits_df=microblog_train_stocktwits_full[['cashtag','sentiment score','spans']].copy()

stocktwits_message= microblog_train_stocktwits_full['message'].apply(pd.Series) #message data
stocktwits_user=stocktwits_message['user'].apply(pd.Series) #user data

stocktwits_df['text']=stocktwits_message['body']
stocktwits_df['created_at']=stocktwits_message['created_at']
stocktwits_df['source']=microblog_train_stocktwits_full['source']
stocktwits_df['official_account']=stocktwits_user['official']
stocktwits_df['sentiment']=stocktwits_message['entities'].apply(pd.Series)['sentiment'].apply(lambda x : x['basic'] if (type(x) is dict) else np.nan) #add sentiment: 'bearish' or 'bullish'
stocktwits_df['liked_by_self']=stocktwits_message['liked_by_self']
stocktwits_df['conversation_parent']=stocktwits_message['conversation'].apply(lambda x : x['parent'] if (type(x) is dict) else np.nan)
stocktwits_df['conversation_replies']=stocktwits_message['conversation'].apply(lambda x : x['replies'] if (type(x) is dict and not math.isnan(x['replies'])) else 0)
stocktwits_df['conversation_replies']=stocktwits_df['conversation_replies'].apply(lambda x: (x-min(stocktwits_df['conversation_replies']))/(max(stocktwits_df['conversation_replies'])-min(stocktwits_df['conversation_replies']))) #min-max scaling
stocktwits_df['total_likes']=stocktwits_message['likes'].apply(lambda x : x['total'] if (type(x) is dict and not math.isnan(x['total'])) else 0)
stocktwits_df['total_likes']=stocktwits_df['total_likes'].apply(lambda x: (x-min(stocktwits_df['total_likes']))/(max(stocktwits_df['total_likes'])-min(stocktwits_df['total_likes']))) #min-max scaling
stocktwits_df['created_at']=pd.to_datetime(stocktwits_df['created_at']).apply(lambda x: '[0am,9am)' 
                                                                              if (x.hour>=0 and x.hour<9) else '[9am,3pm)' 
                                                                              if (x.hour>=9 and x.hour<15) else '[3pm,24pm)') #encoding time created

In [0]:
#convert to numeric
twitter_df['sentiment score']=pd.to_numeric(twitter_df['sentiment score'])
stocktwits_df['sentiment score']=pd.to_numeric(stocktwits_df['sentiment score'])

#One hot encoding
stocktwits_df['official_account']=stocktwits_df['official_account'].apply(lambda x: 1 if x==True else 0 if x==False else x)
stocktwits_df['liked_by_self']=stocktwits_df['liked_by_self'].apply(lambda x: 1 if x==True else 0 if x==False else x)
stocktwits_df['conversation_parent']=stocktwits_df['conversation_parent'].apply(lambda x: 1 if x==True else 0 if x==False else x)

In [0]:
#Concatenate both dataframe
concat_df=pd.concat([stocktwits_df,twitter_df])
concat_df=concat_df[pd.isnull(concat_df.text)==False]
concat_df=concat_df.reset_index()

# Data Cleaning

In [0]:
#Inspired by: 
# https://www.dotnetperls.com/punctuation-python
# https://github.com/tthustla/twitter_sentiment_analysis_part1/blob/master/Capstone_part2.ipynb
# https://github.com/Deffro/text-preprocessing-techniques/blob/master/techniques.py

#Word ordinal encoding
p = inflect.engine()
word_to_number_mapping = {}
for i in range(1, 2000):
  word_form = p.number_to_words(i)  # 1 -> 'one'
  ordinal_word = p.ordinal(word_form)  # 'one' -> 'first'
  ordinal_number = p.ordinal(i)  # 1 -> '1st'
  word_to_number_mapping[ordinal_word] = ordinal_number  # 'first': '1st'
  
  
def elongated_word(word):
    """
     Replaces an elongated word with its basic form, unless the word exists in the lexicon 
     """
    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if (len(word)>2 and word[0] != '$'):#if not Stock Market symbol
      if wn.synsets(word):
          return word
      repl_word = repeat_regexp.sub(repl, word)
      if repl_word != word:      
          return elongated_word(repl_word)
      else:       
          return repl_word
    else:
      return word
    
def isfloat(value):
  ''' 
  Check if value is float or not
  '''
  try:
    float(value)
    return True
  except ValueError:
    return False

def tokenize(sentence):
  '''
  tokenize input sentence into token
  '''
  token_list=nltk.regexp_tokenize(sentence, pattern=r"\s|[\.,;]\D", gaps=True)
  return(token_list)

def sentences_cleaner(sentence):
  '''
  clean input sentence  
  '''
  try:
    mention_pat= r'@[A-Za-z0-9_]+'
    mention_2_pat=r'@[A-Za-z0-9_]+:\s'
    retweet_pat=r'^RT +'
    dollars_pat=r'\$ +'
    http_pat = r'https?://[^ ]+'
    www_pat = r'www.[^ ]+'
    apos_pat=r'"+|"$|"+"$'
      
    soup = BeautifulSoup(sentence, 'lxml')
    souped = soup.get_text()
    
    #HTML decoding remove BOM
    try:
      bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
      bom_removed = souped
          
    #Delete mention
    stripped = re.sub(mention_2_pat,"", bom_removed)
    stripped = re.sub(mention_pat,"", stripped)
      
    #Delete retweet
    stripped=re.sub(retweet_pat,"",stripped)
      
    #Transform any url into '_url'
    stripped = re.sub(http_pat, '_url', stripped)
    stripped = re.sub(www_pat, '_url', stripped)
    
    #Transfrom abbreviation & slang word  into normal words based on abb_dict corpus
    abbreviation_handled=' '.join(pd.Series(stripped.split()).apply(lambda x: abb_dict[x] if x in abb_dict.keys() else x).to_list())
   
    #Transform contracted words into normal words
    contraction_handled =contractions.fix(abbreviation_handled)
      
    #Join the stock symbol
    dollars_handled=re.sub(dollars_pat,'$',contraction_handled)
      
    #Transform elongated words into normal words
    elongated_handled=' '.join(pd.Series(dollars_handled.split()).apply(lambda x: elongated_word(x[:-1])+x[-1] if (x[-1] in string.punctuation and not isfloat(x)) else elongated_word(x) if not isfloat(x) else x))
      
    #Transform ordinal number
    ordinal_handled=' '.join(pd.Series(elongated_handled.split()).apply(lambda x: word_to_number_mapping[x.lower()] if x.lower() in word_to_number_mapping.keys() else x))
      
    #Remove unnecesary apostrophes 
    apos_handled=re.sub(apos_pat,'',ordinal_handled)
      
    #Split Last Word Punctuation
    wordpunct=wordpunct_tokenize(apos_handled)
    if (len(wordpunct[-1])>1 and wordpunct[-1][-1] in string.punctuation and wordpunct[-2] not in string.punctuation) or (wordpunct[-1] in string.punctuation and wordpunct[-2] not in string.punctuation):
      words =tokenize(apos_handled)
      words[-1]=wordpunct[-2]
      words.append(wordpunct[-1])
    else:
      words =tokenize(apos_handled)

    return (" ".join(words)).strip()
  except:
    return sentence

In [13]:
print('BEFORE CLEANING: {}'.format("RT $  FB TOMORROW IS MY FIRST BIG WINNNN!!!! #XOXO Trust me it will go up 50.78%!!! #MYWIN"))
print()
print('AFTER CLEANING: {}'.format(sentences_cleaner("RT $  FB TOMORROW IS MY FIRST BIG WINNNN!!!! #XOXO Trust me it will go up 50.78%!!! #MYWIN")))

BEFORE CLEANING: RT $  FB TOMORROW IS MY FIRST BIG WINNNN!!!! #XOXO Trust me it will go up 50.78%!!! #MYWIN

AFTER CLEANING: $FB TOMORROW IS MY 1st BIG WIN!!!! #XOXO Trust me it will go up 50.78%!!! #MYWIN


In [14]:
print('BEFORE CLEANING: {}'.format(concat_df.loc[81,'text']))
print()
print('AFTER CLEANING: {}'.format(sentences_cleaner(concat_df.loc[81,'text'])))

BEFORE CLEANING: RT @Montisumo Down day for all biotechs Good to load up Be confident. $ONTY $CHTP $AIS $HGSI $DNDN &gt;&gt; Agreed $IBB $XBI all same downness

AFTER CLEANING: Down day for all biotechs Good to load up Be confident $ONTY $CHTP $AIS $HGSI $DNDN >> Agreed $IBB $XBI all same downes


In [15]:
print('BEFORE CLEANING: {}'.format(stocktwits_df.loc[43,'text']))
print()
print('AFTER CLEANING: {}'.format(sentences_cleaner(stocktwits_df.loc[43,'text'])))

BEFORE CLEANING: @joekidd lost 600 in like 3 days for my $C investment. Sure wish bad news came out just a few days earlier. Think I&#39;ll go over to Russia!

AFTER CLEANING: lost 600 in like 3 days for my $C investment Sure wish bad news came out just a few days earlier Think I will go over to Russia !
